<H1>Homework 3, Group 23 - version 6</H1>


<H3> Importing libraries </H3>

In [ ]:
file_path = "/home/jagg/Data/airbnb_data"
#file_path = "airbnb_data"

In [ ]:
from gvf import *

<H3> Read .csv file </H3>

<p> Using pandas to read the .csv file and visualizing it on the Data Frame </p>

In [ ]:
df.head()

<H3> Create .tsv files </H3>

<p>
To create each .tsv file, we iterate the .csv source file, line by line, storing each line of the information in variables, one  for each field. Once the end of the row is reached, we join the information into a new file where the data will be separated by tabulations.  The process will be repeated until we reach the last row of the source document.
</p>

In [ ]:
# Create a new file with tsv extension for each row of the dataset.
for i in range(0, len(df)):

    # Open .tsv file.
    file = open(file_path + "/doc_" + str(i) + ".tsv", "w", encoding = "utf-8")
        
    # Get data from dataframe and saving it into a variable
    average_rate_per_night = str(df["average_rate_per_night"][i])
    bedrooms_count = str(df["bedrooms_count"][i])
    city = str(df["city"][i])
    date_of_listing = str(df["date_of_listing"][i])    
    description = str(df["description"][i])
    latitude = str(df["latitude"][i])
    longitude = str(df["longitude"][i])
    title = str(df["title"][i])
    url = str(df["url"][i])
                
    # Join the fields that will be written in the file
    entry = "	".join([average_rate_per_night, 
                        bedrooms_count,
                        city, 
                        date_of_listing, 
                        description, 
                        latitude, 
                        longitude,
                        title,
                        url
                       ])
    
    
    # Write in the .tsv file
    file.write(entry)
    
    # close the .tsv file
    file.close()

<p>
    We now have a file with .tsv extension for each line of the original .csv document stored in the folder "lairbnb_data".
</p>

<H3> Modify .tsv files </H3>

<p> We are now going to modify the information stored on <b><u>description</u></b> and <b><u>title</u></b> columns respectively.</p>
<p> Once the files are created, line breaks, punctuation marks and unwanted characters are eliminated.</p>

In [ ]:
for i in range(len(df)):

    file = open(file_path + "/doc_" + str(i) + ".tsv", "r",
                encoding = "utf-8")                                        # Only read 
    entry = file.read()                                                    # Storing the information
    file.close()                                                           # Closing the file
    
    entry = entry.split("\t")                                              # Separating with tabs

    description = entry[4]                                                 # Accesing the description info
    description = text_formatting(description)                             # Sending description info to the function
    add_voc(description)                                                   # Sending description info to the Vocabulary
    entry[4] = description                                                 # Receiving the description info modified

    title = entry[7]                                                       # Accesing the title info
    title = text_formatting(title)                                         # Sending title info to the function
    add_voc(title)                                                         # Sending title info to the Vocabulary
    entry[7] = title                                                       # Receiving de title info modified

    entry = "	".join(entry)                                              # Joining the info to put it back in file

    file = open(file_path + "/doc_" + str(i) + ".tsv", "w", 
                encoding = "utf-8")                                        # Modifying the file with the processed data
    file.write(entry)                                                      # Writing in the file
    file.close()                                                           # Closing the file
    

<H3> Build vocabulary </H3> 

In [ ]:
# Sorting and deleting duplicate words
vocabulary = list(sorted(set(vocabulary)))    # The vocabulary variable was created in the in the initialization section

In [ ]:
# In this step we will go through the vocabulary to assign an index to each word.
# At the same time we put them in lower case.

file = open("vocabulary.txt", "w", encoding = "utf-8")            

for i in range(len(vocabulary)):            
    file.write(str(i) + ": " + vocabulary[i].lower() + "\n")
    
file.close()

In [ ]:
# What we do in this step ?
vocabulary_file = open("vocabulary.txt", "r", encoding = "utf-8")
vocabulary = vocabulary_file.read()
vocabulary = vocabulary.split("\n")

<H3>Dictionary Creation</H3>
<p>
    Going through each row of the Data Frame is like inspecting each created tsv file.<br>
If we look for each words of the vocabulary in the columns description and title of the data frame, we will know in how many files each word appears.<br>
Then we will use the index of the vocabulary as index of the dictionary and the index numbers of each row, as the document number for the dictionary.
</p>

In [ ]:
for doc_index in range(len(df)):

    # Open .tsv file.
    file = open(file_path + "/doc_" + str(doc_index) + ".tsv", "r", encoding = "utf-8")
    
    # read entry
    entry = file.read()
    entry = entry.split("\t")
    
    # get description and title
    description = entry[4]
    title = entry[7]
    
    # merge in a single string variable the title and the description
    des_tit = description + " " + title
    
    des_tit = set(des_tit.split(" "))
    
    # for every word in the description and title
    for word in des_tit:
        
        # for every line in the vocabulary
        for i in range(1, len(vocabulary)-1):
            
            # get the vocabulary index and word form the respective line
            term_id = vocabulary[i].split(" ")[0].replace(":", "")
            term = vocabulary[i].split(" ")[1]
                   
            # if it's not yet in the dictionary
            if (word == term) and (term_id not in dictionary):
                dictionary[term_id] = [doc_index]                    # add the term_id and doc index to the dictionary
                break
                
            elif (word == term) and (term_id in dictionary):       # else if it's already in the dictionary
                dictionary[term_id].append(doc_index)                   # append the doc index to the dictionary
                break
    file.close()
    
vocabulary_file.close()

<p>Visualizing the Dictionary</p>

<H2>3.1) Conjunctive query</H2>
<p>Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents: </p>

In [ ]:
query = str(input())
query_copy = query.lower()

In [ ]:
## TEST QUERY

query = text_formatting(query)
query = query.split(" ")
query_copy = query_copy.split(" ")

<p> 
    In this code we implement the query search, using two list, one for the word's index and other the document's index, we found and return the intersection between them.
</p>

In [ ]:
vocabulary_file = open("vocabulary.txt", "r", 
                       encoding = "utf-8")                     # Open the vocabulary file
vocabulary = vocabulary_file.read()                            # Reading 
vocabulary = vocabulary.split("\n")                            # Spliting each word in a new Row

term_id_list = list()                                          # Creating a List for Word's index
doc_list = list()                                              # Creating a List of matching documents

for word in query:                                             # Searching for each word in the query into the vocabulary  
        
    boolean = False
    
    # for every line in the vocabulary
    for i in range(1, len(vocabulary)-1):

        # get the vocabulary index and word form the line
        term_id = vocabulary[i].split(" ")[0].replace(":", "")
        term = vocabulary[i].split(" ")[1]

        if (word == term):
            boolean = True
            term_id_list.append(term_id)
            break
    
    if boolean == False:
        term_id_list = list()
        break
                
vocabulary_file.close()

## print(term_id_list)

for term_id in term_id_list:
    if term_id in dictionary:
        doc_list.append(dictionary[term_id])

<p>
    Presentation of the query result<br>
    Documents that meet the selected criteria 
</p>

In [ ]:
if len(doc_list) > 0:
    
    selected_docs = set(doc_list[0])

    for l in doc_list:
        selected_docs = selected_docs.intersection(set(l))
else:
    selected_docs = list()
    
selected_docs = list(selected_docs)

In [ ]:
df.iloc[selected_docs].drop(labels = ["Unnamed: 0", 
                                      "average_rate_per_night", 
                                      "bedrooms_count", 
                                      "date_of_listing", 
                                      "latitude", 
                                      "longitude"], axis = 1)

<H3> Compute TFIDF and Cosine Similarity </H3> 

The general principle is to compute the distance between the query and each one of the documents. This is done by creating a vector for each one of them. Each component of the vectors corresponds to a word. In the query vector, if the word is contained, the component will be $1$, otherwise it will be $0$. In the documents vectors, the components are the $TFIDF = TF \cdot IDF$ of each word in the given document. The $TF$ (term frequency) is defined as $TF = \frac{\textrm{number of occurences of a word}}{\textrm{total number of words}}$

In [ ]:
# Define new dictionary
dictionary2 = dict()

In [ ]:
vocabulary_file = open("vocabulary.txt", "r", 
                       encoding = "utf-8")                     # Open the vocabulary file
vocabulary = vocabulary_file.read()                            # Reading 
vocabulary = vocabulary.split("\n")

for i in range(1, len(dictionary)+1):

    # get the vocabulary index and word form the line
    term_id = vocabulary[i].split(" ")[0].replace(":", "")
    term = vocabulary[i].split(" ")[1]
    
    # for every document
    for doc_index in range(len(df)):
        
        # open file
        file = open(file_path + "/doc_" + str(doc_index) + ".tsv", "r", 
                    encoding = "utf-8")
        
        # read entry
        entry = file.read()
        entry = entry.split("\t")

        # get description and title
        description = entry[4]
        title = entry[7]

        # merge in a single string variable the title and the description
        des_tit = description + " " + title
        des_tit = des_tit.split(" ")
        
        # compute tfidf
        tfidf = TFIDF(term_id, term, des_tit)
        
        # if it's not yet in the dictionary
        if (term_id not in dictionary2):
            dictionary2[term_id] = [(doc_index, tfidf)]      # add the term_id and (doc index, tfidf) tuple to the dictionary
        elif (term_id in dictionary2):                       # else if it's already in the dictionary
            dictionary2[term_id].append((doc_index, tfidf))  # append the (doc index, tfidf) tuple
        
        # close file
        file.close()
        
# close vocabulary
vocabulary_file.close()

In [ ]:
vocabulary_file = open("vocabulary.txt", "r", 
                       encoding = "utf-8")                     # Open the vocabulary file
vocabulary = vocabulary_file.read()                            # Reading 
vocabulary = vocabulary.split("\n")

v_query = [0]*(len(dictionary)+1)
v_docs = list()

for doc_index in range(len(df)):
    v_docs.append([])
    for j in range(len(dictionary)+1):
        v_docs[doc_index].append(0)


for i in range(1, len(dictionary)+1):

    # Get the vocabulary index and word form the line
    term_id = vocabulary[i].split(" ")[0].replace(":", "")
    term = vocabulary[i].split(" ")[1]

    for word in query:
        if (word == term):
            v_query[int(term_id)] = 1
    
    for doc_index in range(len(df)):
        
        v_docs[doc_index][int(term_id)] = dictionary2[term_id][doc_index][1]

vocabulary_file.close()

In [ ]:
# Convert query vector to numpy array
a_query = np.array(v_query)

# Create a heap of scores and a dictionary of scores
heap = list()
heapq.heapify(heap)
scores_dictionary = dict()

# For every document

for doc_index in range(len(df)):
    
    # Convert document vector to numpy array
    a_doc = np.array(v_docs[doc_index])
    
    # Compute the cosine between the query vector and the document vector
    cos = np.dot(a_query, a_doc)/(np.linalg.norm(a_query)*np.linalg.norm(a_doc))
    
    # Put the result in the dictionary
    scores_dictionary[doc_index] = cos
    heapq.heappush(heap, cos)

In [ ]:
# Set the number k of top documents
k = 10

# Get the ordered list of top_k scores from the heap
top_k = heapq.nlargest(k, heap)

In [ ]:
top_k_docs = list()

# Replace scores in the heap with corresponding documents
for i in range(len(top_k)):
    top_k_docs.append(list(scores_dictionary.keys())[list(scores_dictionary.values()).index(top_k[i])])
    del scores_dictionary[list(scores_dictionary.keys())[list(scores_dictionary.values()).index(top_k[i])]]

In [ ]:
df_results = df.iloc[top_k_docs].drop(labels = ["Unnamed: 0", 
                                           "average_rate_per_night", 
                                           "bedrooms_count", 
                                           "date_of_listing", 
                                           "latitude", 
                                           "longitude"], axis = 1)
df_results.index = list(range(1, k+1))

df_results["scores"] = [round(x,2) for x in top_k]
df_results


<H3>Step 4: Define a new score!</H3>
<p>The general idea is to create a ***scoring function*** and assign a score to each document. The scoring function we chose to use is a weighted sum of three scores, associated to ***distance***, ***number of bedrooms***, and ***price*** (average rate per night).</p>  

<p>The weights have been estimated heuristically, assuming that the distance has a greater impact on the quality of the results than any other parameter, while the contribution of the number of bedrooms and price is almost identical. We therefore chose to assign a weight $w_d = 0.6$ to the distance, $w_b = 0.2$ to the number of bedrooms and $w_p = 0.2$ to the price.</p>

<p>The scoring function for the single variables are a ***negative exponential*** $y = e^{-x/10}$ for the distance (taking the value ***1***, maximum score, when the distance is equal zero, and so the city is exactly the one we were searching for), and a ***gaussian*** $y = e^{-(x-x_{ex})^2}$ for the number of bedrooms and the price (which have a peak value ***1*** corresponding to the exact number of bedrooms or th exact price, while the score decreases if the variables are too high or too low with respect to the exact values searched by the user).</p>

<p>The final scoring function is thus:<br>
$y = w_d \cdot e^{-x/10} + w_b \cdot e^{-(n-n_{ex})^2} + w_p \cdot e^{-(p-p_{ex})^2}$
</p>    

In [ ]:
query = str(input())
query_copy = query.lower()

In [ ]:
query = text_formatting(query)
query = query.split(" ")
query_copy = query_copy.split(" ")

In [ ]:
dictionary3 = dict()
cities_dictionary = dict()

In [ ]:
for doc_index in range(len(df)):

    # Open .tsv file.
    file = open(file_path + "/doc_" + str(doc_index) + ".tsv", "r", encoding = "utf-8")
    
    # read entry
    entry = file.read()
    entry = entry.split("\t")
    
    # get description and title
    average_price_per_night = entry[0]
    bedrooms_count = entry[1]
    city = entry[2]
    latitude = entry[5]
    longitude = entry[6]
    score = 0
    distance = 0
    
    
    
    # update dictionary
    dictionary3[doc_index] = [average_price_per_night, 
                              bedrooms_count, 
                              city, 
                              latitude, 
                              longitude, 
                              distance,
                              score]
    
    city = city.lower()
    if city not in cities_dictionary:
        cities_dictionary[city] = (float(latitude), float(longitude))
    
    # Close .tsv file
    file.close()

In [ ]:
q_price = 0
q_beds = 0
q_coord = (0,0)

# In the query

for i in range(len(query)):
    
    # Get average price per night 
    if "$" in query[i]:
        q_price = float(query[i].replace("$", ""))
        
    # Get bedrooms count
    if query[i] == "bedroom":
        q_beds = int(query[i-1])
        
# Get city coordinates        
for city in cities_dictionary:
    
    city = city.split(" ")
    
    if len(city) == 1:
        for word in query_copy:
            if word == city[0]:
                city = city[0]
            
                q_coord = cities_dictionary[city]
        
    else:
        for i in range(len(query_copy)):
            if (query_copy[i] == city[0]) and (query_copy[i+1] == city[1]):
                city = city[0] + " " + city[1]
            
                q_coord = cities_dictionary[city]

In [ ]:
if q_coord != (0,0):
    w_city = 0.6
else:
    w_city = 0
    
if q_beds != 0:
    w_beds = 0.2
else:
    w_beds = 0
    
if q_price != 0:
    w_price = 0.2
else:
    w_price = 0
    
heap = list()
heapq.heapify(heap)
            
for doc_index in range(len(df)):
    
    d_price = dictionary3[doc_index][0]
    d_price = d_price.replace("$", "")
    d_price = float(d_price)
    
    try:
        d_beds = int(dictionary3[doc_index][1])
    except ValueError:
        d_beds = 0
        
    d_city = dictionary3[doc_index][2]
    d_city = d_city.lower()
    d_coord = cities_dictionary[d_city]

    dist = geodesic(q_coord, d_coord).km
             
    s_city = math.exp(-(dist/10))
    s_beds = math.exp(-(d_beds - q_beds)**2)
    s_price = math.exp(-(d_price - q_price)**2)
    
    score = w_city*s_city + w_beds*s_beds + w_price*s_price
    
    dictionary3[doc_index][5] = dist
    dictionary3[doc_index][6] = score
    
    dictionary3[doc_index] = score
    
    heapq.heappush(heap, score)

In [ ]:
# Set the number k of top documents
k = 10

# Get the ordered list of top_k scores from the heap
top_k = heapq.nlargest(k, heap)

In [ ]:
top_k_docs = list()

# Replace scores in the heap with corresponding documents
for i in range(len(top_k)):
    doc_index = list(dictionary3.keys())[list(dictionary3.values()).index(top_k[i])]
    top_k_docs.append(doc_index)
    del dictionary3[doc_index]

In [ ]:
df_results = df.iloc[top_k_docs].drop(labels = ["Unnamed: 0", 
                                           "average_rate_per_night", 
                                           "bedrooms_count", 
                                           "date_of_listing", 
                                           "latitude", 
                                           "longitude"], axis = 1)
df_results.index = list(range(1, k+1))
df_results.index.name = 'ranking'
df_results